## DCGAN

CelebA Dataset on Kaggle: https://www.kaggle.com/datasets/504743cb487a5aed565ce14238c6343b7d650ffd28c071f03f2fd9b25819e6c9

Paper Link: https://arxiv.org/abs/1511.06434

In [ ]:
print("DCGAN - Deep Convolution GAN : On CelebA Dataset")

In [ ]:
# Clear previous tensorboard logs
%rm -rf logs/

In [ ]:
# make sure to load tensorboard or reload if already loaded 
%load_ext tensorboard

In [ ]:
# start the tensorboard server (default is port 6006)
%tensorboard --logdir=logs/ --reload_interval=5

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),    # 32x32
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4,2,1),     # 16x16
            self._block(features_d * 2, features_d * 4, 4,2,1), # 8x8
            self._block(features_d*4, features_d * 8, 4,2,1),   # 4x4
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),   # to ensure that the valuea are between 0 and 1
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False,),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self,x):
        return self.disc(x)
        

In [ ]:

class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            self._block(z_dim, features_g*16, 4,1,0),   # N x f_g*16 x 4 x 4
            self._block(features_g*16, features_g*8, 4,2,1),    # 8x8
            self._block(features_g*8, features_g*4, 4,2,1),     # 16x16
            self._block(features_g*4, features_g*2, 4,2,1),     # 32x32
            nn.ConvTranspose2d(
                features_g*2, channels_img, kernel_size=4, stride=2, padding=1,
            ),      # 64x64
            nn.Tanh(),  # [-1,1]  
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.gen(x)

In [ ]:
# Mean=0, standatd deviaiton = 0.02
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            

In [ ]:
def test():
    N, in_channels, H, W = 8,3,64,64
    z_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)
    
    gen = Generator(z_dim, in_channels, 8)
    z = torch.randn((N, z_dim, 1, 1))
    initialize_weights(gen)
    
    assert gen(z).shape == (N, in_channels, H, W)
    print("Success")

In [ ]:
test()

### Setup of Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from model import Discriminator, Generator, initialize_weights

Setting up the Hyperparameters ,etc.


In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 3    # make it 1 if you are training on a grayscale image
Z_DIM = 100
NUM_EPOCHS = 5      # It will take a lot of time coz there are lots of images to process
FEATURES_DISC = 64
FEATURES_GEN = 64

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

In [ ]:
# dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms, download=True)
dataset = datasets.ImageFolder(root="celebA_dataset/", transform=transforms)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)

initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, Z_DIM, 1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")

step = 0

## Send both the networks to training modes
print(gen.train())
disc.train()

In [ ]:
for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn((BATCH_SIZE, Z_DIM, 1,1)).to(device)
        fake = gen(noise)

        ### Train the discriminator: max log(D(x)) + log(1-D(G(x)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        loss_disc = (loss_disc_real + loss_disc_fake)/2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train the Generator: min log(1-D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        ###  Print Losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS} Batch {batch_idx}/{len(dataloader)} \n Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}]")

            with torch.no_grad():
                fake = gen(fixed_noise)
                # Take out (upto) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)

                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
            
            step += 1


In [ ]:
%tensorboard --logdir=logs/ --reload_interval=5